In [574]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader,random_split
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau
import torchmetrics.functional as metrics
import os
import shutil
from torchvision import transforms
from PIL import Image

In [575]:
# 학습 검증용 데이터
folder_path = 'data/train/train'
target_data = []
img_data = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    for img in os.listdir(folder_path+'/'+label):
        image_path = os.path.join(folder_path,label,img)
        with open(image_path, 'rb') as file:
            image = Image.open(file)
            # 이미지 크기 확인
            width, height = image.size
            if width == 48 and height == 48:
                image_array = np.array(image)
                
                target_data.append(encoding_label)
                img_data.append(image_array)

In [ ]:
# 테스트용 데이터
folder_path_ = 'data/test/test'
target_test = []
img_test = []
for encoding_label,label in enumerate(os.listdir(folder_path_)):
    for img in os.listdir(folder_path_+'/'+label):
        image_path = os.path.join(folder_path_,label,img)
        with open(image_path, 'rb') as file:
            image = Image.open(file)
            if width == 48 and height == 48:
                image_array = np.array(image)
                target_test.append(encoding_label)
                img_test.append(image_array)

In [581]:
# 이미지 데이터 정규화
x_data = np.array(img_data)/255.
x_data = x_data.reshape((-1,48*48))
print(x_data.shape)

(28709, 2304)


In [582]:
# 원핫 인코딩
# from sklearn.preprocessing import OneHotEncoder
# y_data = OneHotEncoder(sparse_output=False).fit_transform(np.array(target_data).reshape(-1,1))

In [ ]:
# 데이터 클래스 생성
import random
class DLdataset(Dataset):
    
    def __init__(self,x_data,y_data):
        super().__init__()
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self,idx):
        return self.feature[idx], self.target[idx]

In [584]:
# 데이터셋 생성
dataset = DLdataset(x_data,target_data)

In [585]:
# 학습용, 검증용 데이터 준비
seed = torch.Generator().manual_seed(42)
trainDS, validDS = random_split(dataset, [0.8,0.2], generator=seed)

In [586]:
# 배치사이즈 32
BATCH = 32
trainDL = DataLoader(trainDS, batch_size=BATCH)
validDL = DataLoader(validDS, batch_size=BATCH)

In [587]:
# 모델 클래스 정의
class Model(nn.Module):
    
    def __init__(self, IN, OUT, AF):
        super().__init__()
        self.input = nn.Linear(IN, 128) 
        self.af = AF()
        self.hidden = nn.Linear(128, 32)
        self.output = nn.Linear(32, OUT)
        
    def forward(self, x):
        y = self.input(x)
        y = self.af(y)
        y = self.hidden(y)
        y = self.af(y)
        y = self.output(y)
        
        return y

In [588]:
# 학습 준비

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

EPOCHS = 100

IN = dataset.feature.shape[1]
OUT = pd.Series(target_data).nunique()

# 모델 생성
sig_model = Model(IN, OUT, nn.Sigmoid).to(DEVICE)
soft_model = Model(IN, OUT, nn.Identity).to(DEVICE)
relu_model = Model(IN, OUT, nn.ReLU).to(DEVICE)
leaky_model = Model(IN, OUT, nn.LeakyReLU).to(DEVICE)
tanh_model = Model(IN, OUT, nn.Tanh).to(DEVICE)

# 손실함수
LF = nn.CrossEntropyLoss().to(DEVICE)

# 옵티마이저
sig_OPTIMIZER = torch.optim.Adam(sig_model.parameters())
soft_OPTIMIZER = torch.optim.Adam(soft_model.parameters()) # 손실함수에서 soft로
relu_OPTIMIZER = torch.optim.Adam(relu_model.parameters())
leaky_OPTIMIZER = torch.optim.Adam(leaky_model.parameters())
tanh_OPTIMIZER = torch.optim.Adam(tanh_model.parameters())

# 스케줄러
sig_SCHEDULER = ReduceLROnPlateau(sig_OPTIMIZER, mode = 'min', patience = 3)
soft_SCHEDULER = ReduceLROnPlateau(soft_OPTIMIZER, mode = 'min', patience = 3)
relu_SCHEDULER = ReduceLROnPlateau(relu_OPTIMIZER, mode = 'min', patience = 3)
leaky_SCHEDULER = ReduceLROnPlateau(leaky_OPTIMIZER, mode = 'min', patience = 3)
tanh_SCHEDULER = ReduceLROnPlateau(tanh_OPTIMIZER, mode = 'min', patience = 3)


In [589]:
def training(dataLoader, model, optimizer):
    
    model.train()
    train_report=[[], []]
    for (feature, target) in dataLoader:

        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        # 학습
        pre_target = model(feature)
        
        # 손실계산
        loss = LF(pre_target, target)
        train_report[0].append(loss.item())
  
        # 성능 평가
        acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
        train_report[1].append(acc.item())
        
        # W,b업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss_score = sum(train_report[0])/len(train_report[0])
    acc_score = sum(train_report[1])/len(train_report[1])
    print(f'[Train loss] ==> {loss_score}    [Train Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [590]:
def testing(dataLoader, model):
    
    model.eval()
    
    with torch.no_grad():
        val_report=[[], []]
        for (feature, target)  in dataLoader:
            # 배치크기만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            
            # 학습
            pre_target = model(feature)
            
            # 손실계산
            loss = LF(pre_target, target)
            val_report[0].append(loss.item())
      
            # 성능 평가
            acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
            val_report[1].append(acc.item())
    
        loss_score = sum(val_report[0])/len(val_report[0])
        acc_score = sum(val_report[1])/len(val_report[1])

    print(f'[Test loss] ==> {loss_score}    [Test Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [591]:
def predicting(dataset,n,Model,filename):
# 0:angry / 1:disgust / 2:fear / 3:happy / 4:neutral / 5:sad / 6:surprise
    label = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
    model_path = 'model/'+filename
    Model.load_state_dict(torch.load(model_path))
    Model.eval()

    correct = 0
    total = 0

    for idx in range(len(dataset)):
        img, ytrue = dataset[idx][0], dataset[idx][1]

        with torch.no_grad():
            ypre = Model(img.unsqueeze(0))
            ypre = torch.argmax(ypre, dim=1).item()

            if img.shape[0] == 2304:
                img = img.reshape(48, 48)

            if ypre == ytrue:
                correct += 1
            
            total += 1
            if idx < n :
                plt.imshow(dataset[idx][0].numpy().reshape(48, 48), cmap='gray_r')
                plt.title(f'[{idx+1}] True {label[ytrue]} / Predict {label[ypre]}')
                plt.xticks([])
                plt.yticks([])
                plt.show()


    accuracy = correct / total
    print(f'Accuracy: {accuracy:.2f}, Correct : {correct}, Total : {total}')
    return accuracy

In [592]:
def grahp(train_report,val_report):
    fig, ax1 = plt.subplots()  # fig는 그래프 객체

    # 손실 그래프 그리기
    ax1.plot(train_report[0], label='Train Loss', color='red')
    ax1.plot(val_report[0], label='Validation Loss', color='orange')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax1.tick_params(axis='y')
    ax1.legend(loc='upper left')
    
    # 정확도 그래프 그리기
    ax2 = ax1.twinx()
    ax2.plot(train_report[1], label='Train Accuracy', color='blue')
    ax2.plot(val_report[1], label='Validation Accuracy', color='green')
    ax2.set_ylabel('Accuracy & F1')
    ax2.set_xlabel('Epoch')
    ax2.tick_params(axis='y')
    ax2.legend(loc='upper right')
    
    plt.xlabel('Epoch')  # 전체 그래프의 x축 라벨 설정
    plt.title('Train & Validation Scores')
    plt.show()

In [597]:
def model_tv(TrainDL,ValidDL,Model,Optimizer, Epoch ,Scheduler, filename):
    dir = 'model/'
    filepath=dir+filename
    if not os.path.exists(dir):
        os.makedirs(dir)
    min_loss = 100.0  # 초기 최소 손실 설정
    train_report = [[],[]]
    val_report = [[],[]]
    for eps in range(Epoch):
        print(f'[{eps+1}/{Epoch}]')
        # 학습
        train_loss, train_acc = training(TrainDL,Model,Optimizer)
        train_report[0].append(train_loss)
        train_report[1].append(train_acc)
        
        # 검증
        val_loss, val_acc = testing(ValidDL,Model)
        val_report[0].append(val_loss)
        val_report[1].append(val_acc)
        # 최소 손실 업데이트
        if val_loss < min_loss:
            min_loss = val_loss
            torch.save(Model.state_dict(), filepath)
    
        # 조기 종료 기능 => 조건 : val_loss가 지정된 횟수 이상 개선이 안되면 학습 종료
        if Scheduler.num_bad_epochs >= Scheduler.patience:
            print(f"Early stopping at epoch {eps}")
            break
    return train_report,val_report        

In [598]:
relu_train_,relu_val_ = model_tv(trainDL,validDL,relu_model,relu_OPTIMIZER,EPOCHS,relu_SCHEDULER,'relu_model.pth')

[1/100]
[Train loss] ==> 1.1670819096744558    [Train Accuracy] ==> 0.5503568941504178
[Test loss] ==> 2.194105465544595    [Test Accuracy] ==> 0.34571314106384915
[2/100]
[Train loss] ==> 1.1733310860179593    [Train Accuracy] ==> 0.5451195450048261
[Test loss] ==> 2.1971652097172205    [Test Accuracy] ==> 0.339383012884193
[3/100]
[Train loss] ==> 1.1634039648894148    [Train Accuracy] ==> 0.5544481197771588
[Test loss] ==> 2.2793932874997456    [Test Accuracy] ==> 0.3242922008865409
[4/100]
[Train loss] ==> 1.1621747894373444    [Train Accuracy] ==> 0.5493268338252575
[Test loss] ==> 2.328128809399075    [Test Accuracy] ==> 0.33739316239953043
[5/100]
[Train loss] ==> 1.1583783116015218    [Train Accuracy] ==> 0.5554346564254389
[Test loss] ==> 2.290954866674211    [Test Accuracy] ==> 0.33600427351064155
[6/100]
[Train loss] ==> 1.1527308751945708    [Train Accuracy] ==> 0.5565227483474444
[Test loss] ==> 2.3317427655061085    [Test Accuracy] ==> 0.3376602564420965
[7/100]
[Train lo

In [599]:
sig_train_,sig_val_ = model_tv(trainDL,validDL,sig_model,sig_OPTIMIZER,EPOCHS,sig_SCHEDULER,'sig_model.pth')

[1/100]
[Train loss] ==> 1.8132358912305913    [Train Accuracy] ==> 0.2499854921215424
[Test loss] ==> 1.803421660926607    [Test Accuracy] ==> 0.2509214743971825
[2/100]
[Train loss] ==> 1.7817841613857195    [Train Accuracy] ==> 0.2638695450463335
[Test loss] ==> 1.7342305342356363    [Test Accuracy] ==> 0.2905982906619708
[3/100]
[Train loss] ==> 1.734130388680939    [Train Accuracy] ==> 0.3076108403109574
[Test loss] ==> 1.6986876255936092    [Test Accuracy] ==> 0.3109909188416269
[4/100]
[Train loss] ==> 1.7164191118854002    [Train Accuracy] ==> 0.32182857473082527
[Test loss] ==> 1.6894626988304986    [Test Accuracy] ==> 0.3355368591017193
[5/100]
[Train loss] ==> 1.7063268056154914    [Train Accuracy] ==> 0.32578922932692556
[Test loss] ==> 1.6813592420683967    [Test Accuracy] ==> 0.34352297021283046
[6/100]
[Train loss] ==> 1.697368055831091    [Train Accuracy] ==> 0.3302721680455885
[Test loss] ==> 1.6775161504745484    [Test Accuracy] ==> 0.3451789531442854
[7/100]
[Train l

In [600]:
tanh_train_,tanh_val_ = model_tv(trainDL, validDL, tanh_model, tanh_OPTIMIZER, EPOCHS, tanh_SCHEDULER, 'tanh_model.pth')

[1/100]
[Train loss] ==> 1.8152913507644846    [Train Accuracy] ==> 0.2497678737371413
[Test loss] ==> 1.807718943225013    [Test Accuracy] ==> 0.2509214743971825
[2/100]
[Train loss] ==> 1.8059987072160981    [Train Accuracy] ==> 0.25620937791541426
[Test loss] ==> 1.7535261339611476    [Test Accuracy] ==> 0.28720619661940466
[3/100]
[Train loss] ==> 1.7647274090054972    [Train Accuracy] ==> 0.27785515320334264
[Test loss] ==> 1.751314288377762    [Test Accuracy] ==> 0.2872863247990608
[4/100]
[Train loss] ==> 1.750523624811996    [Train Accuracy] ==> 0.2833246286117931
[Test loss] ==> 1.735151513417562    [Test Accuracy] ==> 0.29015758550829357
[5/100]
[Train loss] ==> 1.7452054251203297    [Train Accuracy] ==> 0.29062209840770553
[Test loss] ==> 1.7255892720487382    [Test Accuracy] ==> 0.29935897439718245
[6/100]
[Train loss] ==> 1.743859755627624    [Train Accuracy] ==> 0.2887070566249757
[Test loss] ==> 1.7221462329228718    [Test Accuracy] ==> 0.3004006410638491
[7/100]
[Train 

In [ ]:
leaky_train_,leaky_val_ = model_tv(trainDL,validDL,leaky_model,leaky_OPTIMIZER,EPOCHS,leaky_SCHEDULER,'leaky_model.pth')

[1/100]
[Train loss] ==> 1.7829820510404688    [Train Accuracy] ==> 0.2715732358541329
[Test loss] ==> 1.7207419958379533    [Test Accuracy] ==> 0.30169604702128305
[2/100]
[Train loss] ==> 1.7130896804392504    [Train Accuracy] ==> 0.3181000464390911
[Test loss] ==> 1.67884880039427    [Test Accuracy] ==> 0.3368589743971825
[3/100]
[Train loss] ==> 1.681341701231295    [Train Accuracy] ==> 0.33542246983741986
[Test loss] ==> 1.6475000805324977    [Test Accuracy] ==> 0.3478766025768386
[4/100]
[Train loss] ==> 1.6604857318580648    [Train Accuracy] ==> 0.34739148097948114
[Test loss] ==> 1.6311501450008816    [Test Accuracy] ==> 0.3588141025768386
[5/100]
[Train loss] ==> 1.6427448531379274    [Train Accuracy] ==> 0.3544858403109574
[Test loss] ==> 1.624993599123425    [Test Accuracy] ==> 0.3605502136879497
[6/100]
[Train loss] ==> 1.625951111316681    [Train Accuracy] ==> 0.36301648097948114
[Test loss] ==> 1.6110702070924972    [Test Accuracy] ==> 0.3699252136879497
[7/100]
[Train lo

In [ ]:
soft_train_,soft_val_ = model_tv(trainDL,validDL,soft_model,soft_OPTIMIZER,EPOCHS,soft_SCHEDULER,'soft_model.pth')

In [ ]:
grahp(relu_train_,relu_val_)

In [ ]:
grahp(tanh_train_,tanh_val_)

In [ ]:
grahp(leaky_train_,leaky_val_)

In [ ]:
grahp(soft_train_,soft_val_)

In [ ]:
grahp(sig_train_,sig_val_)

In [ ]:
# 이미지 데이터 정규화
x_test = np.array(img_test)/255.
x_test = x_test.reshape((-1,48*48))

In [ ]:
testDS = DLdataset(x_test,target_test)

In [ ]:
testDL = DataLoader(testDS, batch_size=BATCH, drop_last=True, shuffle=True)

In [ ]:
predicting(testDL,10,relu_model,'relu_model.pth')

In [ ]:
predicting(testDL, 10, sig_model, 'sig_model.pth')

In [ ]:
predicting(testDL, 10, tanh_model, 'tanh_model.pth')

In [ ]:
predicting(testDL, 10, leaky_model, 'leaky_model.pth')

In [ ]:
predicting(testDL, 10, soft_model, 'soft_model.pth')